<a href="https://colab.research.google.com/github/shivster2000/UROPs2025/blob/main/Bot-to-Bot_ControllableComplexityChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Controllable Complexity Chatbot from Tyen (2022)

 - Connect to a GPU runtime for optimal inference times
 - Run all below cells, and the chatbot will load in the output of the demo.py cell

# Install Dependencies

In [1]:
!git clone https://github.com/shivster2000/ControllableComplexityChatbot.git

Cloning into 'ControllableComplexityChatbot'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 69 (delta 28), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (69/69), 1.13 MiB | 4.85 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [2]:
cd /content/ControllableComplexityChatbot

/content/ControllableComplexityChatbot


In [3]:
pip install numpy scipy regex torch transformers iopath

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for iopath: filename=iopath-0.1.10-py3-none-any.whl size=31527 sha256=190dfb68fe88bae2dba58c093e108c0cb87d135e246e3f595af6cb8cb2d7a52d
  Stored in directory: /root/.cache/pip/wheels/7c/96/04/4f5f31ff812f684f69f40cb1634357812220aac58d4698048c
Successfully built iopath


#Install ParlAI

In [4]:
!git clone https://github.com/facebookresearch/ParlAI.git --branch 1.6.0

Cloning into 'ParlAI'...
remote: Enumerating objects: 48019, done.
remote: Total 48019 (delta 0), reused 0 (delta 0), pack-reused 48019 (from 1)
Receiving objects: 100% (48019/48019), 145.65 MiB | 19.28 MiB/s, done.
Resolving deltas: 100% (34129/34129), done.
Note: switching to '054a0fff8183e357727dc7a91682496734badb7f'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false



In [5]:
!pip install git+https://github.com/facebookresearch/ParlAI.git@1.6.0

  Cloning https://github.com/facebookresearch/ParlAI.git (to revision 1.6.0) to /tmp/pip-req-build-fymhowu6
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/ParlAI.git /tmp/pip-req-build-fymhowu6
  Running command git checkout -q 054a0fff8183e357727dc7a91682496734badb7f
  Resolved https://github.com/facebookresearch/ParlAI.git to commit 054a0fff8183e357727dc7a91682496734badb7f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.3/266.3 kB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 59.3 MB/s eta 0:00:00
  Installing build depe

The below cell **will** give an error - ignore this; the chatbot seems not to run properly without running this command

In [6]:
!cd ParlAI; python setup.py develop; cd ..

running develop
/usr/local/lib/python3.12/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://github.com/pypa/setuptools/issues/917 for details.
        ********************************************************************************

!!
  easy_install.initialize_options(self)
/usr/local/lib/python3.12/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer or other
        standards-based tools.

        See https://blog

In [7]:
%cd /content/ControllableComplexityChatbot/

/content/ControllableComplexityChatbot


#Run Demo

In [1]:
import json

%cd /content/ControllableComplexityChatbot/ParlAI
from parlai.zoo.blender.blender_3B import download
from parlai.core.opt import Opt

%cd /content/ControllableComplexityChatbot/
from controllable_blender import ControllableBlender

agent_opt = json.load(open("blender_3B.opt", 'r'))

# Set to "vocab" for vocabulary restriction, "rerank" for candidate reranking
agent_opt["inference"] = "rerank"
# Same top-k sampling configs for all settings described in the paper
agent_opt["beam_size"] = 20
agent_opt["topk"] = 40

# Settings for rerank methods (not used if "inference" == "vocab")
agent_opt["rerank_cefr"] = "B1"                       # CEFR level to adjust reranking. Possible values: ['A2', 'B1', 'B2', 'C1', 'C2'].
agent_opt["rerank_tokenizer"] = "distilroberta-base"        # Tokenizer from Huggingface Transformers. Must be compatible with "rerank_model"
agent_opt["rerank_model"] = "complexity_model"              # Model fine-tuned on complexity data
agent_opt["rerank_model_device"] = "cuda"                   # Device for complexity model
agent_opt["penalty_stddev"] = 2                             # Controls how harshly sub-tokens are penalised (lower = harsher). Use -1 to remove penalties
agent_opt["filter_path"] = "data/filter.txt"                # Path to list of English words to ensure OOV words are not generated. Capitalised words are ignored. Use empty string to remove filter

# Settings for vocab methods (not used if "inference" == "rerank")
agent_opt["wordlist_path"] = "data/sample_wordlist.txt"          # Path to list of vocab the chatbot is restricted to


download(agent_opt["datapath"])

agent = ControllableBlender(agent_opt)

#import baby model
from transformers import AutoTokenizer, LlamaForCausalLM
from parlai.core.agents import Agent
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

#climb-mao/babylm-B1-fine-tuned

#instantiate your model here
class TransformersAgentWrapper(Agent):
    def __init__(self, opt, model_name="climb-mao/babylm-A-fine-tuned", max_new_tokens=60):
        super().__init__(opt)
        self.id = "A-Learner"
        self.device = "cuda" if torch.cuda.is_available() else "cpu"

        # Load tokenizer + model
        self.tokenizer = AutoTokenizer.from_pretrained(
            model_name,
            trust_remote_code=True
        )
        if self.tokenizer.pad_token is None:
            self.tokenizer.pad_token = self.tokenizer.eos_token

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=True
        ).to(self.device)

        self.max_new_tokens = 60
        self.context = ""

    def observe(self, observation):
        if 'episode_done' not in observation:
            observation = observation.copy()  # avoid mutating input dict
            observation['episode_done'] = False
        self.observation = observation
        if not observation.get("episode_done", False):
            speaker = "Teacher" if observation.get("from") == "teacher" else "Student"
            self.context += f"\n{speaker}: {observation.get('text', '')}"

    def act(self):
        system_instruction = "You are a student. Respond in a maximum of two short sentences."
        prompt = f"{self.context}\nStudent:"
        inputs = self.tokenizer(
            system_instruction + "\n" + prompt,
            return_tensors="pt",
            truncation=True,
            max_length=512
        ).to(self.device)

        outputs = self.model.generate(
            **inputs,
            max_new_tokens=self.max_new_tokens,
            pad_token_id=self.tokenizer.pad_token_id,
            eos_token_id=self.tokenizer.eos_token_id,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            temperature=0.7
        )

        full_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        reply = full_text.split("Student:")[-1].strip()

        # Trim to max 2 sentences
        sentences = reply.replace("\n", " ").split(". ")
        reply = ". ".join(sentences[:2]).strip()
        if not reply.endswith("."):
            reply += "."

        self.context += f"\nStudent: {reply}"
        return {"id": self.id, "text": reply}

from parlai.core.params import ParlaiParser
from parlai.core.worlds import DialogPartnerWorld
from controllable_blender import ControllableBlender

class TwoBotDialogPartnerWorld(DialogPartnerWorld):
    def __init__(self, opt, agent1, agent2, max_turns=5):
        super().__init__(opt, [agent1, agent2])
        self.turns = 0
        self.max_turns = max_turns

    def parley(self):
        acts = []
        for idx, agent in enumerate(self.agents):
            # First agent's turn
            if idx == 0 and self.turns == 0:
                text = "The teacher greets you. Respond as the student."
                obs = {"text": text, "episode_done": False, "from": "teacher"}
            else:
                # Use the last message from the other agent
                text = acts[-1]['text'] if acts else ""
                obs = {"text": text, "episode_done": False, "from": "teacher" if idx == 1 else "student"}

            obs = {"text": text, "episode_done": False}
            agent.observe(obs)
            acts.append(agent.act())

        self.acts = acts
        for m in acts:
            print(f"{m['id']}: {m.get('text', '')}")

        self.turns += 1

    def episode_done(self):
        return self.turns >= self.max_turns


if __name__ == "__main__":
    parser = ParlaiParser()
    opt = parser.parse_args([])

    # Instantiate agents
    hf_agent = TransformersAgentWrapper(opt, model_name="climb-mao/babylm-A-fine-tuned")
    parlai_agent = agent

    # Create and run world
    world = TwoBotDialogPartnerWorld(opt, hf_agent, parlai_agent, max_turns=5)

    while not world.episode_done():
        world.parley()

/content/ControllableComplexityChatbot/ParlAI
/content/ControllableComplexityChatbot
23:39:22 INFO | Using CUDA
23:39:22 INFO | loading dictionary from ParlAI/data/models/blender/blender_3B/model.dict
23:39:22 INFO | num words = 8008
23:40:11 INFO | Total parameters: 2,696,268,800 (2,695,613,440 trainable)
23:40:11 INFO | Loading existing model params from ParlAI/data/models/blender/blender_3B/model
A-Learner: the teacher's work. Also, there are a lot of interesting things for them.
ControllableBlender: I have heard that in some countries teachers are highly respected, which is a good thing! 
A-Learner: the book, the school and the history of the old students. Interests and activities: play basketball, and play the piano.scenter: Get some music at the cinema and watch a movie.
ControllableBlender: That's very true! The role of teacher varies a lot across the world, too. 
A-Learner: Im really enjoy the people, it is a great way to me!: a bit boring time, we can know how to play somethi

# Below Script deprecated manual version

In [ ]:
import json

%cd /content/ControllableComplexityChatbot/ParlAI
from parlai.zoo.blender.blender_3B import download
from parlai.core.opt import Opt

%cd /content/ControllableComplexityChatbot/
from controllable_blender import ControllableBlender
from demo_utils import start_interaction

agent_opt = json.load(open("blender_3B.opt", 'r'))

# Set to "vocab" for vocabulary restriction, "rerank" for candidate reranking
agent_opt["inference"] = "rerank"
# Same top-k sampling configs for all settings described in the paper
agent_opt["beam_size"] = 20
agent_opt["topk"] = 40

# Settings for rerank methods (not used if "inference" == "vocab")
agent_opt["rerank_cefr"] = "B2"                       # CEFR level to adjust reranking. Possible values: ['A2', 'B1', 'B2', 'C1', 'C2'].
agent_opt["rerank_tokenizer"] = "distilroberta-base"        # Tokenizer from Huggingface Transformers. Must be compatible with "rerank_model"
agent_opt["rerank_model"] = "complexity_model"              # Model fine-tuned on complexity data
agent_opt["rerank_model_device"] = "cuda"                   # Device for complexity model
agent_opt["penalty_stddev"] = 2                             # Controls how harshly sub-tokens are penalised (lower = harsher). Use -1 to remove penalties
agent_opt["filter_path"] = "data/filter.txt"                # Path to list of English words to ensure OOV words are not generated. Capitalised words are ignored. Use empty string to remove filter

# Settings for vocab methods (not used if "inference" == "rerank")
agent_opt["wordlist_path"] = "data/sample_wordlist.txt"          # Path to list of vocab the chatbot is restricted to


download(agent_opt["datapath"])

agent = ControllableBlender(agent_opt)

#import baby model
from transformers import AutoTokenizer, LlamaForCausalLM

model = LlamaForCausalLM.from_pretrained("timinar/baby-llama-58m")
tokenizer = AutoTokenizer.from_pretrained("timinar/baby-llama-58m")

student_persona = (
    "You are a curious student who loves talking about travel and music."
)
print("\n==== Starting 5-Turn Interaction ====\n")

for turn in range(1, 6):
    print(f"\n--- Turn {turn} ---")

    if turn == 1:
        # Start with an opening for Baby LLaMA
        llama_prompt = (
            f"{student_persona}\nThe teacher greets you. Respond as the student."
        )
    else:
        # Craft a new prompt using last Blender reply
        llama_prompt = (
            f"{student_persona}\n"
            f"Teacher just said: \"{blender_response}\"\n"
            "Reply as the student with curiosity or sharing your own view."
        )

    # Baby LLaMA generates a prompt
    inputs = tokenizer(llama_prompt, return_tensors="pt", truncation=True, max_length=256)
    input_ids = inputs.input_ids

    gen_ids = model.generate(
        input_ids,
        max_length=input_ids.shape[1] + 80,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8,
        eos_token_id=tokenizer.encode("\nTeacher:", add_special_tokens=False)[0]
    )
    generated_tokens = gen_ids[0][input_ids.shape[1]:]
    student_prompt = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    print(f"Baby LLaMA: {student_prompt}")

    # Controllable Blender responds
    agent.observe({"text": student_prompt, "episode_done": False})
    blender_output = agent.act()

if "text" not in blender_output:
    print(f"[DEBUG] No text in Blender output: {blender_output}")
    blender_response = "[No response generated]"
else:
    blender_response = blender_output["text"]

print(f"ControllableBlender: {blender_response}")

/content/ControllableComplexityChatbot/ParlAI
/content/ControllableComplexityChatbot


KeyboardInterrupt: 